In [11]:
GROUND_TRUTH_PATH = 'songs20190129.csv'
LABEL_FILE_PATH = 'song_labels.pkl'
SONG_BASEPATH = 'sound/'
MELSPEC_BASEPATH = 'melspec/'
TEMPO_TOLERANCE = .05
HOPLEN=256

In [12]:
import librosa
import numpy as np
import pandas as pd
import scipy
import pickle
import gzip
import os

In [13]:
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib nbagg
from IPython.display import Audio

In [14]:
paces = pd.read_csv(GROUND_TRUTH_PATH, usecols=range(4), index_col=0, header=None)
paces.index.name = 'song'
paces.columns = ['pace0', 'pace1', 'pace2']
paces.head()

,pace0,pace1,pace2
song,,,
03 Death of a Disco Dancer,172.0,86.0,43.0
04 Black Horse and the Cherry Tree,103.0,207.0,NaN
09 Suddenly I See,99.0,198.0,NaN
53rdAnd3rdMP,137.0,68.0,273.0
ActNaturally,92.0,186.0,NaN


In [15]:
def is_compatible(song, tempo, truth=paces, tol=TEMPO_TOLERANCE):
    if song in truth.index:
        return(np.any(np.isclose(tempo, truth.loc[song].dropna().values, rtol=tol)))
    else:
        print( 'Song not found: ', song )
        return(False)

In [16]:
basepath = SONG_BASEPATH
suffix = '.mp3'
songlist = [f[:-4] for f in os.listdir(basepath) if f[-4:]==suffix and f[:-4] in paces.index]

In [17]:
verbose = False
nstart, nend, hoplen, chunklen = 15, 132, HOPLEN, 2**19
song_tempos = {}
for song in songlist:
    tempo_set = set()
    songpath = basepath + song + suffix
    print( songpath )
    y, sr = librosa.load(songpath)
    splits = list(range(chunklen,len(y)-chunklen,chunklen))
    for start, end in zip(splits[:-1], splits[1:]):
        tempos = librosa.feature.tempogram(y[start:end], sr=sr, hop_length=hoplen)
        tempo_func = np.mean(tempos,axis=1)[nstart:nend]
        tempo_list = sr*60/(hoplen*np.array(range(nstart,nend)))
        peaks, peaks_dict = scipy.signal.find_peaks(tempo_func)
        if verbose: print( '\n', start, end, song )
        for tempo in tempo_list[peaks]:
            compatible = is_compatible(song, tempo)
            candidate = (int(round(tempo)), compatible)
            if verbose: print(candidate)
            tempo_set.add(candidate)
    if verbose: print(tempo_set)
    song_tempos[song] = tempo_set
    melspec_path = MELSPEC_BASEPATH+song+'_melspec.pkl.gz'
    if not os.path.exists(melspec_path):
        melspec = librosa.feature.melspectrogram(y[chunklen:-chunklen], hop_length=HOPLEN)
        with gzip.open(MELSPEC_BASEPATH+song+'_melspec.pkl.gz', 'wb') as fp:
            pickle.dump(melspec.astype('float32'), fp)
if verbose: print('\n', song_tempos)

sound/03 Death of a Disco Dancer.mp3
sound/04 Black Horse and the Cherry Tree.mp3
sound/09 Suddenly I See.mp3
sound/53rdAnd3rdMP.mp3
sound/ActNaturally.mp3
sound/AHardRainsAGonnaFall.mp3
sound/AintAGonnaBeTreatedThisaway.mp3
sound/Allentown.mp3
sound/AllMyLoving.mp3
sound/Amore.mp3
sound/AndWeDancedMP.mp3
sound/AnotherGirl.mp3
sound/becausethenightcd.mp3
sound/BeMineTonight.mp3
sound/BeWithYou1mp.mp3
sound/BigJimMP.mp3
sound/BloodyMerryMorning.mp3
sound/BlueMoon.mp3
sound/BothSidesNow.mp3
sound/Byrds - Hey Mr. Tambourine Man.mp3
sound/CaliforniaDreamin2MP.mp3
sound/CallMe1.mp3
sound/CharlieBrown.mp3
sound/ChatanoogaChooChoo.mp3
sound/Chicago Blues Harmonica Poject - Dusty Brown He Don't Love You.mp3
sound/Collide.mp3
sound/ComeToMyWindow.mp3
sound/CreequeAlley.mp3
sound/Criminal.mp3
sound/CrocodileRock1.mp3
sound/DirkBalthausTrioMelAndChloe.mp3
sound/DontBotherMe.mp3
sound/DontCryMP.mp3
sound/DontPassMeBy.mp3
sound/DontSetMeFree.mp3
sound/DontWaitTillTomorrowMP.mp3
sound/DoReMi.mp3
sou

In [18]:
with open(LABEL_FILE_PATH, 'wb') as fp:
    pickle.dump(song_tempos, fp)